In [1]:
import bs4
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
# Base URL
base_url = 'https://www.atticsalt.in/collections/all-products?page='

# Initialize variables
page = 1
all_products = []
all_highlight = []
all_price = []
all_reviews = []

while True:
    url = base_url + str(page)
    data = requests.get(url)
    web_data = BeautifulSoup(data.text, 'html.parser')

    # Extract all info
    products = [item.text.strip() for item in web_data.find_all('a', {'class': 'product-title h6 line-clamp'})]
    highlight = [item.text.strip() for item in web_data.find_all('span', {'class': 'badge badge--custom'})]
    price = [item.text.strip() for item in web_data.find_all('sale-price', {'class': 'h6 text-on-sale'})]
    reviews = [item.text.strip() for item in web_data.find_all('span', {'class': 'smallcaps text-xxs text-subdued'})]

    # Break the loop if no products are found
    if not products:
        break

    # Fill missing values with 'N/A' to ensure the lengths are equal
    # Fill missing highlight, price, and reviews if they are shorter than the products list
    highlight += ['N/A'] * (len(products) - len(highlight))
    price += ['N/A'] * (len(products) - len(price))
    reviews += ['N/A'] * (len(products) - len(reviews))

    # Now all lists should have the same length, so we can add them to the final lists
    all_products.extend(products)
    all_highlight.extend(highlight)
    all_price.extend(price)
    all_reviews.extend(reviews)

    # Increment to the next page
    page += 1


In [101]:
# To make data frame

# Zip the data into a list of tuples
zipped_data = list(zip(all_products, all_highlight,all_price,all_reviews))

# Create a DataFrame from the zipped data
atticsalt = pd.DataFrame(zipped_data, columns=["Product Name", "Highlight","Price","Reviews"])
atticsalt

,Product Name,Highlight,Price,Reviews
0,Butterfly Majesty Black Shirt,EXCLUSIVE,"Sale priceFrom Rs. 3,465.00",(21)
1,So- Sexy Multi Tier Pink Ruffle Dress,BEST SELLER,"Sale priceFrom Rs. 5,225.00",(55)
2,Pearl White Statement Opulent Sleeve Blouse,READY TO SHIP,Sale priceFrom Rs. 748.00,(17)
3,Contrast Flower Embroidery Mini,BEST SELLER,"Sale priceFrom Rs. 6,626.75",(22)
4,Shimmer It Real Embellished Dress,READY TO SHIP,"Sale priceFrom Rs. 2,260.00",(16)
...,...,...,...,...
1548,Disco night black sequins dress,N/A,"Sale priceFrom Rs. 4,916.25",N/A
1549,Cape Style Embroidered Hem Dress,N/A,"Sale priceRs. 5,265.00",N/A
1550,Black Lace Mini Dress,N/A,"Sale priceFrom Rs. 2,430.00",N/A
1551,Black Stones Embellished Bodysuit,N/A,Sale priceRs. 946.00,N/A


In [4]:
# TheLabelLife.com Data Frame

In [152]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

# Set up the Chrome driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Open the URL
driver.get('https://store.thelabellife.com/collections/clothing')

# Scroll to load more products (you can adjust the scroll behavior as needed)
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)  # Wait for new products to load
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# Now that all content is loaded, you can scrape the data
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Extract the product names and prices
products = [item.text.strip() for item in soup.find_all('div', {'class': 'text-subtext5 mb-1 truncate uppercase sm:text-body3 sm:font-bold'})]
prices = [item.text.strip() for item in soup.find_all('div', {'class': 'flex flex-row space-x-2 items-center'})]

# Handle any missing data
if len(products) > len(prices):
    prices += ['N/A'] * (len(products) - len(prices))
elif len(prices) > len(products):
    products += ['N/A'] * (len(prices) - len(products))

# Print or save the results
#print(products)
#print(prices)

# Close the driver
driver.quit()


In [153]:
# To make data frame

# Zip the data into a list of tuples
zipped_data = list(zip(products, prices))

# Create a DataFrame from the zipped data
labellife = pd.DataFrame(zipped_data, columns=["Product Name","Price"])
labellife

,Product Name,Price
0,Black Shimmer Bow Tie Up Dress,"MRP₹3,990"
1,Grey Shimmer Soft Fringe Cami,"MRP₹2,390"
2,Blue Shimmer Bow Neck Dress,"MRP₹3,490"
3,Grey Shimmer Soft Fringe Skirt,"MRP₹2,390"
4,Beige Sequin One Shoulder Dress,"MRP₹3,690"
...,...,...
575,Black Floral Tiered Midi Dress,"MRP₹2,390₹1,79325% OFF"
576,Mocha Midi Dress,"MRP₹1,990₹1,49325% OFF"
577,Aqua Square Neck Top,"MRP₹1,990₹1,49325% OFF"
578,Lilac Strappy Tie Up Dress,"MRP₹3,490₹1,92045% OFF"


In [7]:
# FableStreet

In [8]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

# Set up the Chrome driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Open the URL
driver.get('https://www.fablestreet.com/collections/the-formal-edit')

# Scroll to load more products (you can adjust the scroll behavior as needed)
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)  # Wait for new products to load
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# Now that all content is loaded, you can scrape the data
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Extract the product names and prices
products = [item.text.strip() for item in soup.find_all('p', {'class': 'MuiTypography-root MuiTypography-p mui-q7n1p6'})]
prices = [item.text.strip() for item in soup.find_all('span', {'class': 'MuiTypography-root MuiTypography-span mui-1aiep3i'})]

# Handle any missing data
if len(products) > len(prices):
    prices += ['N/A'] * (len(products) - len(prices))
elif len(prices) > len(products):
    products += ['N/A'] * (len(prices) - len(products))

# Print or save the results
#print(products)
#print(prices)

# Close the driver
driver.quit()


In [9]:
# To make data frame

# Zip the data into a list of tuples
zipped_data = list(zip(products, prices))

# Create a DataFrame from the zipped data
fablestreet = pd.DataFrame(zipped_data, columns=["Product Name","Price"])
fablestreet

,Product Name,Price
0,LivIn Bootcut Pants - Black,"₹ 1,885"
1,Satin Stripes Shirt - Pink,"₹ 1,745"
2,A-line Jacket Dress - White,"₹ 4,995"
3,Crew Neck Top - Pink,"₹ 1,595"
4,Cotton Colour Block Tee - Black And White,"₹ 1,395"
...,...,...
479,Satin Wrap Top - Teal,"₹ 1,995"
480,Square Neck Ribbed Tee - Navy,"₹ 1,355"
481,Abstract Print Shirt - Orange,"₹ 2,155"
482,Boat Neck Fit And Flare Dress - Black,"₹ 2,875"


In [269]:
# AJIO

In [270]:
url = 'https://www.ajio.com/s/latest-collections-4461-75481?query=%3Arelevance%3Agenderfilter%3AWomen&curated=true&curatedid=latest-collections-4461-75481&gridColumns=3&segmentIds='
data = requests.get(url)
web_data = BeautifulSoup(data.text, 'html.parser')

from selenium import webdriver
from bs4 import BeautifulSoup

# Set up Selenium
driver = webdriver.Chrome()
driver.get(url)

# Wait for the page to load (if needed)
driver.implicitly_wait(10)

# Get the page source and parse it with BeautifulSoup
html = driver.page_source
web_data = BeautifulSoup(html, 'html.parser')

# Find the desired elements
name = web_data.find_all('div', {'class': 'nameCls'})
prices = web_data.find_all('span', {'class': 'price'})

# Close the driver
driver.quit()
products = [item['aria-label'] for item in name if 'aria-label']




In [368]:
# To make data frame

# Zip the data into a list of tuples
zipped_data = list(zip(products, prices))

# Create a DataFrame from the zipped data
ajio = pd.DataFrame(zipped_data, columns=["Product Name","Price"])
ajio

,Product Name,Price
0,Woman Embroidery Straight Kurta Set,"[[₹1,899]]"
1,Floral Print Flared Kurta Set,[[₹699]]
2,Women Striped Regular Fit Sweatshirt,[[₹399]]
3,Women Pack of 2 Ribbed Turtleneck Pullovers,[[₹850]]
4,Women Flared Track Pants,"[[₹2,999]]"
5,Floral Straight Kurta Set,"[[₹2,024]]"
6,Women Colourblock Sneakers with Lace Fastening,"[[₹1,160]]"
7,Women Regular Fit Bombers Jacket,[[₹750]]
8,Women Typographic Print Regular Fit Hoodie,[[₹760]]
9,Women Kurta Set,[[₹700]]


In [13]:
# #  Nykaa Fashion

In [14]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

# Set up the Chrome driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Open the URL
driver.get('https://www.nykaafashion.com/recently-added/c/4878?root=nav_3&ptype=listing%2Cwomen%2Cwhat%27s-new%2Cjust-in%2C2%2C&f=gender_filter%3D5196_&isFilteredUrl=true&p=4')

# Scroll to load more products (you can adjust the scroll behavior as needed)
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)  # Wait for new products to load
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# Now that all content is loaded, you can scrape the data
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Extract the product names and prices
products = [item.text.strip() for item in soup.find_all('div', {'class': 'css-1f9lck7'})]
prices = [item.text.strip() for item in soup.find_all('div', {'class': 'css-1oh7ecm'})]

# Handle any missing data
if len(products) > len(prices):
    prices += ['N/A'] * (len(products) - len(prices))
elif len(prices) > len(products):
    products += ['N/A'] * (len(prices) - len(products))


# Close the driver
driver.quit()


In [15]:
# To make data frame

# Zip the data into a list of tuples
zipped_data = list(zip(products, prices))

# Create a DataFrame from the zipped data
df = pd.DataFrame(zipped_data, columns=["Product Name","Price"])
df

,Product Name,Price
0,Women Brown Casual Boots,Get it for ₹1850
1,Gold Tone Floral Pearls and Crystal Wedding Pa...,Get it for ₹679


In [16]:
# the database

In [17]:
atticsalt

,Product Name,Highlight,Price,Reviews
0,Butterfly Majesty Black Shirt,EXCLUSIVE,"Sale priceFrom Rs. 3,465.00",(21)
1,So- Sexy Multi Tier Pink Ruffle Dress,BEST SELLER,"Sale priceFrom Rs. 5,225.00",(55)
2,Pearl White Statement Opulent Sleeve Blouse,READY TO SHIP,Sale priceFrom Rs. 748.00,(17)
3,Contrast Flower Embroidery Mini,BEST SELLER,"Sale priceFrom Rs. 6,626.75",(22)
4,Shimmer It Real Embellished Dress,READY TO SHIP,"Sale priceFrom Rs. 2,260.00",(16)
...,...,...,...,...
1548,Disco night black sequins dress,N/A,"Sale priceFrom Rs. 4,916.25",N/A
1549,Cape Style Embroidered Hem Dress,N/A,"Sale priceRs. 5,265.00",N/A
1550,Black Lace Mini Dress,N/A,"Sale priceFrom Rs. 2,430.00",N/A
1551,Black Stones Embellished Bodysuit,N/A,Sale priceRs. 946.00,N/A


In [18]:
labellife

,Product Name,Price
0,Black Shimmer Bow Tie Up Dress,"MRP₹3,990"
1,Grey Shimmer Soft Fringe Cami,"MRP₹2,390"
2,Blue Shimmer Bow Neck Dress,"MRP₹3,490"
3,Grey Shimmer Soft Fringe Skirt,"MRP₹2,390"
4,Beige Sequin One Shoulder Dress,"MRP₹3,690"
...,...,...
595,Olive Belted Midi Dress,"MRP₹2,290₹1,71825% OFF"
596,Orange Off Shoulder Midi Dress,"MRP₹2,890₹1,59045% OFF"
597,Red & Green Stripe Detail Shirt,"MRP₹2,790₹1,39550% OFF"
598,Floral Patchwork Shirt Dress,"MRP₹2,590₹1,94325% OFF"


In [19]:
fablestreet

,Product Name,Price
0,LivIn Bootcut Pants - Black,"₹ 1,885"
1,Satin Stripes Shirt - Pink,"₹ 1,745"
2,A-line Jacket Dress - White,"₹ 4,995"
3,Crew Neck Top - Pink,"₹ 1,595"
4,Cotton Colour Block Tee - Black And White,"₹ 1,395"
...,...,...
479,Satin Wrap Top - Teal,"₹ 1,995"
480,Square Neck Ribbed Tee - Navy,"₹ 1,355"
481,Abstract Print Shirt - Orange,"₹ 2,155"
482,Boat Neck Fit And Flare Dress - Black,"₹ 2,875"


In [20]:
ajio

,Product Name,Price
0,Woman Embroidery Straight Kurta Set,"[[₹1,899]]"
1,Floral Print Flared Kurta Set,[[₹699]]
2,Women Striped Regular Fit Sweatshirt,[[₹399]]
3,Women Regular Fit Jacket,[[₹750]]
4,Women Pack of 2 Ribbed Turtleneck Pullovers,[[₹850]]
5,Women Low-Top Sneakers with Lace Fastening,"[[₹1,507]]"
6,Women Flared Track Pants,"[[₹2,999]]"
7,Women Colourblock Sneakers with Lace Fastening,"[[₹1,160]]"
8,Women Regular Fit Bombers Jacket,[[₹750]]
9,Women Typographic Print Regular Fit Hoodie,[[₹760]]


In [21]:
# for data cleaning

In [25]:
atticsalt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1553 entries, 0 to 1552
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Product Name  1553 non-null   object
 1   Highlight     1553 non-null   object
 2   Price         1553 non-null   object
 3   Reviews       1553 non-null   object
dtypes: object(4)
memory usage: 48.7+ KB


In [30]:
atticsalt['Highlight'].unique()

array(['EXCLUSIVE', 'BEST SELLER', 'READY TO SHIP', 'SALE', 'BESTSELLER',
       'NEW ARRIVAL', 'Best Seller', 'N/A', 'TRENDING', 'NEWARRIVAL'],
      dtype=object)

In [26]:
atticsalt.head(2)

,Product Name,Highlight,Price,Reviews
0,Butterfly Majesty Black Shirt,EXCLUSIVE,"Sale priceFrom Rs. 3,465.00",(21)
1,So- Sexy Multi Tier Pink Ruffle Dress,BEST SELLER,"Sale priceFrom Rs. 5,225.00",(55)


In [31]:
atticsalt['Highlight'].nunique()

10

In [38]:
atticsalt['Highlight'].value_counts()

Highlight
NEW ARRIVAL      523
N/A              467
READY TO SHIP    444
EXCLUSIVE         37
SALE              32
BEST SELLER       27
NEWARRIVAL        20
BESTSELLER         1
Best Seller        1
TRENDING           1
Name: count, dtype: int64

In [48]:
atticsalt['Price']

0       Sale priceFrom Rs. 3,465.00
1       Sale priceFrom Rs. 5,225.00
2         Sale priceFrom Rs. 748.00
3       Sale priceFrom Rs. 6,626.75
4       Sale priceFrom Rs. 2,260.00
                   ...             
1548    Sale priceFrom Rs. 4,916.25
1549         Sale priceRs. 5,265.00
1550    Sale priceFrom Rs. 2,430.00
1551           Sale priceRs. 946.00
1552         Sale priceRs. 3,870.00
Name: Price, Length: 1553, dtype: object

In [102]:
atticsalt["Price"] = atticsalt["Price"].apply(lambda x: x.split("Rs. ")[-1].strip())

In [103]:
atticsalt['Price'] = atticsalt['Price'].str.replace(',', '')

In [104]:
atticsalt["Price"] = atticsalt["Price"].apply(lambda x: x.split("₹")[-1].strip())

In [105]:
atticsalt['Price'] = atticsalt['Price'].str.replace('N/A', '0')

In [106]:
atticsalt['Price'] = atticsalt['Price'].astype(float)

In [107]:
atticsalt['Reviews']

0       (21)
1       (55)
2       (17)
3       (22)
4       (16)
        ... 
1548     N/A
1549     N/A
1550     N/A
1551     N/A
1552     N/A
Name: Reviews, Length: 1553, dtype: object

In [108]:
atticsalt['Reviews'] = atticsalt['Reviews'].str.replace('(', '').str.replace(')', '').str.replace('N/A', '0')

In [110]:
atticsalt['Reviews'] = atticsalt['Reviews'].astype(float)

In [112]:
atticsalt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1553 entries, 0 to 1552
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Product Name  1553 non-null   object 
 1   Highlight     1553 non-null   object 
 2   Price         1553 non-null   float64
 3   Reviews       1553 non-null   float64
dtypes: float64(2), object(2)
memory usage: 48.7+ KB


In [113]:
atticsalt.head(2)

,Product Name,Highlight,Price,Reviews
0,Butterfly Majesty Black Shirt,EXCLUSIVE,3465.0,21.0
1,So- Sexy Multi Tier Pink Ruffle Dress,BEST SELLER,5225.0,55.0


In [118]:
highest_price = atticsalt.sort_values(by = 'Price', ascending = False)

In [119]:
highest_price

,Product Name,Highlight,Price,Reviews
1201,Sparkling studded Black Cord Set,NEW ARRIVAL,18050.0,1.0
1134,Olive Seductress Slit Cut Dress,NEW ARRIVAL,13110.0,0.0
1211,Hand - Crafted White Cord- Set,NEW ARRIVAL,12967.5,0.0
541,Ever-Pretty Embellished Navy Gown,EXCLUSIVE,10700.0,1.0
583,Attic Curves Ever-Pretty Embellished Navy Gown,READY TO SHIP,10212.5,1.0
...,...,...,...,...
1334,NAVY MUSIC TEE,N/A,0.0,0.0
1335,Pop White Blazer,N/A,0.0,0.0
1336,PLAIN BLACK TEE,N/A,0.0,0.0
1337,PATCHED PETALS PRETTY PINK,N/A,0.0,0.0


In [120]:
highest_reviews = atticsalt.sort_values(by = 'Reviews', ascending = False)

In [138]:
atticsalt[atticsalt['Reviews']>9].shape

(20, 4)

In [139]:
atticsalt['Highlight'].value_counts()

Highlight
NEW ARRIVAL      523
N/A              467
READY TO SHIP    444
EXCLUSIVE         37
SALE              32
BEST SELLER       27
NEWARRIVAL        20
BESTSELLER         1
Best Seller        1
TRENDING           1
Name: count, dtype: int64

In [140]:
# for labellife data

In [155]:
labellife["Price"] = labellife["Price"].apply(lambda x: x.split("Rs. ")[-1].strip())

In [156]:
labellife["Price"].value_counts()

Price
MRP₹1,990₹1,49325% OFF    14
MRP₹2,990₹2,24325% OFF    13
MRP₹2,490₹1,49440% OFF    12
MRP₹2,390₹1,79325% OFF    11
MRP₹2,590₹1,94325% OFF    10
                          ..
MRP₹3,590₹2,33435% OFF     1
MRP₹1,940₹77660% OFF       1
MRP₹1,590₹63660% OFF       1
MRP₹2,140₹1,39135% OFF     1
MRP₹3,590₹1,43660% OFF     1
Name: count, Length: 215, dtype: int64

In [157]:
labellife

,Product Name,Price
0,Black Shimmer Bow Tie Up Dress,"MRP₹3,990"
1,Grey Shimmer Soft Fringe Cami,"MRP₹2,390"
2,Blue Shimmer Bow Neck Dress,"MRP₹3,490"
3,Grey Shimmer Soft Fringe Skirt,"MRP₹2,390"
4,Beige Sequin One Shoulder Dress,"MRP₹3,690"
...,...,...
575,Black Floral Tiered Midi Dress,"MRP₹2,390₹1,79325% OFF"
576,Mocha Midi Dress,"MRP₹1,990₹1,49325% OFF"
577,Aqua Square Neck Top,"MRP₹1,990₹1,49325% OFF"
578,Lilac Strappy Tie Up Dress,"MRP₹3,490₹1,92045% OFF"


In [161]:
labellife_2 = labellife # only for saving the data for backup

In [159]:
labellife['Price'] = labellife['Price'].str.extract(r'₹(\d{1,},\d{3})(?!.*₹)')

In [160]:
labellife

,Product Name,Price
0,Black Shimmer Bow Tie Up Dress,"3,990"
1,Grey Shimmer Soft Fringe Cami,"2,390"
2,Blue Shimmer Bow Neck Dress,"3,490"
3,Grey Shimmer Soft Fringe Skirt,"2,390"
4,Beige Sequin One Shoulder Dress,"3,690"
...,...,...
575,Black Floral Tiered Midi Dress,"1,793"
576,Mocha Midi Dress,"1,493"
577,Aqua Square Neck Top,"1,493"
578,Lilac Strappy Tie Up Dress,"1,920"


In [172]:
labellife.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 580 entries, 0 to 579
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Product Name  580 non-null    object
 1   Price         526 non-null    object
dtypes: object(2)
memory usage: 9.2+ KB


In [170]:
labellife['Price'].nunique()

181

In [193]:
labellife['Price'] = labellife['Price'].str.replace(',', '').astype(float)


In [197]:
labellife['Price'].value_counts()

Price
1493.0    14
2243.0    13
1494.0    12
1793.0    11
2691.0    10
          ..
1603.0     1
4290.0     1
1391.0     1
2030.0     1
1436.0     1
Name: count, Length: 181, dtype: int64

In [198]:
labellife.sort_values(by = 'Price', ascending = False)

,Product Name,Price
198,Silver Sequinned Panel Dress,5890.0
247,Gold Shimmer One Shoulder Dress,5490.0
262,Ice Shimmer Twist Knot Jumpsuit,4690.0
319,Rose Gold Ruched Maxi Dress,4690.0
226,Ice Shimmer Kaftan Sleeve Dress,4290.0
...,...,...
501,Black Crinkled Pants,NaN
511,Black Tie Up Strap Dress,NaN
512,Checkered Mini Skirt,NaN
515,Navy Ruffle Sleeve Midi Dress,NaN


In [201]:
labellife[labellife['Price']>5000]

,Product Name,Price
198,Silver Sequinned Panel Dress,5890.0
247,Gold Shimmer One Shoulder Dress,5490.0


In [206]:
labellife[labellife['Price']>1000]

,Product Name,Price
0,Black Shimmer Bow Tie Up Dress,3990.0
1,Grey Shimmer Soft Fringe Cami,2390.0
2,Blue Shimmer Bow Neck Dress,3490.0
3,Grey Shimmer Soft Fringe Skirt,2390.0
4,Beige Sequin One Shoulder Dress,3690.0
...,...,...
575,Black Floral Tiered Midi Dress,1793.0
576,Mocha Midi Dress,1493.0
577,Aqua Square Neck Top,1493.0
578,Lilac Strappy Tie Up Dress,1920.0


In [204]:
labellife.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 580 entries, 0 to 579
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Product Name  580 non-null    object 
 1   Price         526 non-null    float64
dtypes: float64(1), object(1)
memory usage: 9.2+ KB


In [214]:
labellife[labellife['Price'].isna()].shape

(54, 2)

In [216]:
526 + 54

580

In [220]:
labellife[labellife['Price']>1000]

,Product Name,Price
0,Black Shimmer Bow Tie Up Dress,3990.0
1,Grey Shimmer Soft Fringe Cami,2390.0
2,Blue Shimmer Bow Neck Dress,3490.0
3,Grey Shimmer Soft Fringe Skirt,2390.0
4,Beige Sequin One Shoulder Dress,3690.0
...,...,...
575,Black Floral Tiered Midi Dress,1793.0
576,Mocha Midi Dress,1493.0
577,Aqua Square Neck Top,1493.0
578,Lilac Strappy Tie Up Dress,1920.0


In [221]:
labellife[labellife['Price']<1000]

,Product Name,Price


In [222]:
# fablestreet

In [227]:
fablestreet_2 = fablestreet # for making a duplicate data

In [234]:
fablestreet.sample(30)

,Product Name,Price
16,Front Pleated Top - White,"₹ 1,915"
421,Single Breasted Sleeveless Blazer - Black,"₹ 3,675"
319,Satin Wrap Top - Navy,"₹ 1,425"
203,Satin Geometric Print Shift Dress - Grey,"₹ 2,085"
291,Cotton Pleat Detail Top - Blush Pink,"₹ 1,295"
310,LivIn Single Breasted Blazer - Black,"₹ 3,675"
142,Lapel Collar Shirt - Navy Blue,"₹ 1,835"
182,Single Breasted Blazer - Black,"₹ 4,745"
132,LivIn Button Down Cotton Knit Shirt - Black,"₹ 1,325"
414,Cowl Neck Polka Top - Maroon,"₹ 1,035"


In [229]:
fablestreet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 484 entries, 0 to 483
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Product Name  484 non-null    object
 1   Price         484 non-null    object
dtypes: object(2)
memory usage: 7.7+ KB


In [235]:
fablestreet['Price'] = fablestreet['Price'].str.extract(r'(\d{1,3}(,\d{3})*)')[0].str.replace(',', '').astype(int)

In [238]:
fablestreet.head(5)

,Product Name,Price
0,LivIn Bootcut Pants - Black,1885
1,Satin Stripes Shirt - Pink,1745
2,A-line Jacket Dress - White,4995
3,Crew Neck Top - Pink,1595
4,Cotton Colour Block Tee - Black And White,1395


In [239]:
# for creating a new column for colour

In [240]:
fablestreet['Colour'] = fablestreet['Product Name'].str.extract(r'-\s*(.+)$')

In [241]:
fablestreet

,Product Name,Price,Colour
0,LivIn Bootcut Pants - Black,1885,Black
1,Satin Stripes Shirt - Pink,1745,Pink
2,A-line Jacket Dress - White,4995,line Jacket Dress - White
3,Crew Neck Top - Pink,1595,Pink
4,Cotton Colour Block Tee - Black And White,1395,Black And White
...,...,...,...
479,Satin Wrap Top - Teal,1995,Teal
480,Square Neck Ribbed Tee - Navy,1355,Navy
481,Abstract Print Shirt - Orange,2155,Orange
482,Boat Neck Fit And Flare Dress - Black,2875,Black


In [242]:
fablestreet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 484 entries, 0 to 483
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Product Name  484 non-null    object
 1   Price         484 non-null    int32 
 2   Colour        483 non-null    object
dtypes: int32(1), object(2)
memory usage: 9.6+ KB


In [255]:
fablestreet['Colour'].value_counts()[:15]

Colour
Black              61
Navy Blue          26
White              24
Black And White    21
Beige              21
Grey               20
Maroon             18
Fuchsia            15
Teal               14
Olive              12
Blue               12
Navy               12
White And Black    11
Off White           9
Green               9
Name: count, dtype: int64

In [253]:
fablestreet.sort_values(by  = 'Price', ascending = False)[:20]

,Product Name,Price,Colour
432,Linen Co-ord Set - Light Blue,6445,ord Set - Light Blue
297,Single Breasted Blazer And Trousers Co-ord Set...,6370,ord Set - Off White
317,Single Breasted Blazer And Trousers Co-ord Set...,6070,ord Set - Green
433,Linen Co-ord Set - Dusty Pink,6015,ord Set - Dusty Pink
298,Linen Co-ord Set - Navy,5585,ord Set - Navy
409,Linen Co-ord Set - White,5470,ord Set - White
245,Linen Co-ord Set - Beige,5470,ord Set - Beige
84,Linen Co-ord Set - Navy Blue,5470,ord Set - Navy Blue
262,Checkered Blazer And Skirt Co-ord - Black And ...,5355,ord - Black And White
411,Linen Co-ord Set - Light Blue,5105,ord Set - Light Blue


In [251]:
fablestreet[fablestreet['Colour'] == 'Black'].sort_values(by = 'Price', ascending = False)

,Product Name,Price,Colour
182,Single Breasted Blazer - Black,4745,Black
315,Single Breasted Blazer - Black,4395,Black
395,LivIn Double Breasted Blazer - Black,3995,Black
310,LivIn Single Breasted Blazer - Black,3675,Black
421,Single Breasted Sleeveless Blazer - Black,3675,Black
...,...,...,...
269,Essential Work Trousers - Black,1160,Black
266,LivIn Tummy Tuck Pants - Black,1040,Black
296,Floral Print Top - Black,1035,Black
175,Satin Lace Camisole - Black,920,Black


In [256]:
# ajio

In [369]:
ajio.head(5)

,Product Name,Price
0,Woman Embroidery Straight Kurta Set,"[[₹1,899]]"
1,Floral Print Flared Kurta Set,[[₹699]]
2,Women Striped Regular Fit Sweatshirt,[[₹399]]
3,Women Pack of 2 Ribbed Turtleneck Pullovers,[[₹850]]
4,Women Flared Track Pants,"[[₹2,999]]"


In [370]:
ajio['Price'] = ajio['Price'].astype(str)

In [371]:
ajio.head(4)

,Product Name,Price
0,Woman Embroidery Straight Kurta Set,"<span class=""price""><strong>₹1,899</strong></s..."
1,Floral Print Flared Kurta Set,"<span class=""price""><strong>₹699</strong></span>"
2,Women Striped Regular Fit Sweatshirt,"<span class=""price""><strong>₹399</strong></span>"
3,Women Pack of 2 Ribbed Turtleneck Pullovers,"<span class=""price""><strong>₹850</strong></span>"


In [372]:
ajio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Product Name  45 non-null     object
 1   Price         45 non-null     object
dtypes: object(2)
memory usage: 852.0+ bytes


In [373]:
ajio['Price'] = ajio['Price'].str.extract(r'₹([\d,]+)', expand=False)

In [375]:
ajio['Price'] = ajio['Price'].str.replace(',', '')

In [378]:
ajio['Price'] = ajio['Price'].astype(int)

In [380]:
ajio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Product Name  45 non-null     object
 1   Price         45 non-null     int32 
dtypes: int32(1), object(1)
memory usage: 672.0+ bytes


In [383]:
ajio.sort_values(by = 'Price', ascending = False)

,Product Name,Price
18,Women Court Vision LO NN Lace-Up Sneakers,4296
39,Women Regular Fit Single-Breasted Blazer,4129
22,Women Downshifter 13 Low-Top Running Shoes,3737
19,Women Cable-Knit High-Neck Cotton Pullover,3499
41,Women Shoulder Bag with Chain Strap,3399
15,Women Regular Fit Hooded Puffer Jacket,3009
4,Women Flared Track Pants,2999
29,Women Muslin 2-Piece Sets,2430
23,Women Embroidered 2-Piece Kurta & Pants Set,2107
11,Women Leheriya Print Straight Kurta Set,2090


In [388]:
ajio[ajio['Price']>1000].sort_values(by = 'Price', ascending = False).shape

(21, 2)